In [1]:
### 常规分析包
import os
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, pearsonr, spearmanr
from scipy.spatial import distance
### 绘图包
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # 输出PDF文字可编辑
### 其他
import cloudpickle as pickle
import warnings
warnings.filterwarnings("ignore") # 报错提示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

In [2]:
tcgsa = pd.read_csv('../Results/2.1.1. TcGSA_ASV_rel(0.1S4P5EC).csv', index_col=0)
tcgsa.shape
tcgsa.head(2)

ClusrMeds = pd.read_csv('../Results/2.1.1. TcGSA_ASV_rel(0.1S4P5EC)_ClusrMeds.csv')
ClusrMeds.shape
ClusrMeds.head(2)

data = pd.read_csv('../Data/16S/MZJ_dada2_table_ASV.tsv', sep='\t', index_col=0, skiprows=1)
data.head(2)

(310, 28)

,1,2,3,4,5,6,7,8,9,10,...,19,20,21,22,23,24,25,26,27,Cluster
3303dcf065483fbf1a02ff00a59b7848,0.868041,1.127654,1.338262,1.482767,1.500521,1.372479,1.173388,0.980821,0.869184,0.881701,...,2.381668,2.446145,2.533824,2.708566,2.864747,2.879094,2.617276,2.061066,1.327369,1
44c9f59231dd703269388032da1d5a98,0.896473,1.143516,1.341858,1.474706,1.481714,1.343840,1.134613,0.930088,0.803492,0.798238,...,2.240409,2.313456,2.406466,2.582573,2.741246,2.765766,2.528348,2.013547,1.330654,1


(27, 7)

,Unnamed: 0,TimePoint,1,2,3,4,5
0,1,1,-0.684115,-0.174532,-1.636056,-0.562766,-0.360451
1,2,2,-0.750570,0.476257,-0.954842,-1.130979,-0.024060


,C11_10,C1_11,C1_12,C1_13,C1_14,C1_15,C1_16,C1_2,C1_21,C1_22,...,M21_M,M25_M,M26_M,M28_M,M29_M,M31_M,M32_M,M34_M,M35_M,M4_M
#OTU ID,,,,,,,,,,,,,,,,,,,,,
3303dcf065483fbf1a02ff00a59b7848,0,0,39,55,43,0,206,923,883,147,...,0,0,0,0,0,0,78,0,0,0
44c9f59231dd703269388032da1d5a98,0,0,59,42,0,25,185,969,773,111,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from sklearn.decomposition import PCA, IncrementalPCA, TruncatedSVD, NMF
childs = [i for i in data.columns if i[0]=='C']
new_data = pd.DataFrame(index=childs, columns=['Cluster'+str(i) for i in range(1, 6)])
for c in range(1, 6):
    asvs = tcgsa.loc[tcgsa['Cluster']==c, :].index
    temp = data.loc[asvs, childs]
    pca = PCA(n_components=1)
    new_data['Cluster'+str(c)] = pca.fit_transform(temp.T)[:, 0]
    c, temp.shape, pca.explained_variance_ratio_
new_data.head(2)

(1, (88, 450), array([0.46386657]))

(2, (122, 450), array([0.36217977]))

(3, (41, 450), array([0.58007896]))

(4, (11, 450), array([0.97810961]))

(5, (48, 450), array([0.34660769]))

,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5
C11_10,-945.522747,-374.174859,-300.155767,-189.135109,-16.510366
C1_11,-917.347621,-386.517485,-269.337439,-9.402844,-16.817262


In [4]:
meta = pd.read_csv('../Data_preprocess/1.1.MetaData.tsv', sep='\t', index_col=0)
meta = meta.loc[childs, ['Gender', 'Mode of delivery', 'Solid food', 'Breastfeeding', 
                         'Feeding mode(BFD:0,MFD:1)', 'Sibling(Y:1,N:0)', 'PAE(Y:1,N:0)', 'Allergy(Y:1,N:0)']]
meta.columns = ['Gender', 'Delivery', 'SolidFood', 'Breastfeeding', 'FeedingMode', 'Sibling', 'PAE', 'Allergy']
meta.head(2)

,Gender,Delivery,SolidFood,Breastfeeding,FeedingMode,Sibling,PAE,Allergy
ID,,,,,,,,
C11_10,Female,C-section,0.0,1.0,0.0,0.0,0.0,1.0
C1_11,Male,Vaginal,0.0,1.0,1.0,0.0,1.0,1.0


In [5]:
### rpy2
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.pandas2ri import py2rpy, rpy2py
pandas2ri.activate()

In [6]:
importr('mvpart')
importr('MVPARTwrap')

rpy2.robjects.packages.Package as a <module 'mvpart'>

rpy2.robjects.packages.Package as a <module 'MVPARTwrap'>

In [7]:
_ = r.assign('species', new_data)
_ = r.assign('env', meta)

In [8]:
r('''
set.seed(123)
pdf("test.pdf")
spider_mvpart <- mvpart(data.matrix(species)~., env, xval = nrow(species), xvmult = 100, xv = 'pick', pca = TRUE)
spider_mvpart
dev.off();
''')

X-Val rep : 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100
Minimum tree sizes
tabmins
 7  8  9 10 
59  1  3 37 


array([1], dtype=int32)

In [9]:
r('''
set.seed(123)
pdf("test.pdf")
spider_mvpart <- mvpart(data.matrix(species)~., env, xval = nrow(species), xvmult = 100, xv = 'min', pca = TRUE)
spider_mvpart
dev.off();
''')

X-Val rep : 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100
Minimum tree sizes
tabmins
 7  8  9 10 
59  1  3 37 


array([1], dtype=int32)

In [10]:
_ = r('summary(spider_mvpart)')

Call:
mvpart(form = data.matrix(species) ~ ., data = env, xv = "min", 
    xval = nrow(species), xvmult = 100, pca = TRUE)
  n= 450 

          CP nsplit rel error    xerror       xstd
1 0.05906661      0 1.0000000 1.0046399 0.09227083
2 0.05448146      1 0.9409334 0.9709403 0.08867007
3 0.02796658      3 0.8319705 0.8591541 0.07280414
4 0.02398731      4 0.8040039 0.8549457 0.07200069
5 0.02221872      5 0.7800166 0.8411165 0.07129574
6 0.01202595      6 0.7577979 0.8062701 0.06862840

Node number 1: 450 observations,    complexity param=0.05906661
  Means=1.061e-13,-5.065e-14,-5.811e-14,1.388e-13,3.245e-15, Summed MSE=4510541 
  left son=2 (281 obs) right son=3 (169 obs)
  Primary splits:
      Delivery  splits as  LR,      improve=0.059066610, (0 missing)
      Allergy   < 0.5 to the left,  improve=0.026923220, (0 missing)
      Sibling   < 0.5 to the left,  improve=0.023826670, (20 missing)
      SolidFood < 0.5 to the right, improve=0.010096280, (0 missing)
      PAE       < 0.5 t

In [11]:
print(r('''
spider_mvpart_wrap <- MRT(spider_mvpart, percent = 10, species = colnames(species))
spider_mvpart_wrap
'''))

$nodes
[1] 1 2 5 3 6 7

$pourct
      Cluster1   Cluster2     Cluster3    Cluster4     Cluster5
[1,] 70.675781  4.2931284 1.257315e+01 12.45784660 8.950198e-05
[2,] 14.664255 16.2781935 6.899044e+01  0.06408311 3.031725e-03
[3,]  6.676863  2.6524526 9.027366e+01  0.39329909 3.723389e-03
[4,] 79.134978  7.3041705 1.981517e-01 13.31928972 4.341024e-02
[5,] 19.331722 78.5902289 3.157209e-01  1.70446625 5.786185e-02
[6,] 42.936684  0.8733705 9.113194e-04 56.18893585 9.880861e-05

$R2
            [,1]         [,2]         [,3]         [,4]         [,5]
[1,] 0.041745791 0.0025358056 7.426537e-03 7.358428e-03 5.286579e-08
[2,] 0.004101091 0.0045524546 1.929427e-02 1.792186e-05 8.478699e-07
[3,] 0.001601600 0.0006362519 2.165422e-02 9.434186e-05 8.931407e-07
[4,] 0.031408662 0.0028990243 7.864638e-05 5.286424e-03 1.722952e-05
[5,] 0.004295261 0.0174617410 7.014914e-05 3.787105e-04 1.285616e-05
[6,] 0.029743497 0.0006050093 6.312976e-07 3.892372e-02 6.844762e-08

$obj
n= 450 

node), split, n, 